# Model_selection 모듈

### 학습/테스트 데이터 세트 분리 – train_test_split()
- 사용되는 인수들
  * test_size: 디폴트는 0.25
  * train_size : 1-test_size이므로 의미가 없음
  * shuffle: 데이터를 분리하기 전에 데이터를 미리 섞을지를 결정. 디폴트는 True
  * stratify: 학습 데이터의 y 값의 분포를 균등하게 만들어줌
  * random_state : 이 값이 동일하다면 동일한 난수를 사용해서 데이터를 추출함
- 반환 값: 튜플 형태
  * 학습용 데이터의 피쳐 데이터 세트(X_train)
  * 테스트용 데이터의 피쳐 데이터 세트(X_test)
  * 학습용 데이터의 레이블 데이터 세트(y_train)
  * 테스트용 데이터의 레이블 데이터 세트(y_test)

- 훈련 데이터와 테스트 데이터로 분리하지 않고 머신러닝을 수행한다면?
  * 모델의 정확도를 측정할 수가 없음! (정확히 표현하자면 측정한 정확도가 필요없는 수치가 되어버림)
  * 이미 훈련에 사용된 데이터를 테스트에 또 사용하므로 모델의 분류 능력이 아닌 단순히 어떤 독립수에 대한 종속변수를 외워서 내놓는 꼴이 되어버리기 때문


In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [1]:
iris=load_iris()
dtc=DecisionTreeClassifier(random_state=2022)
dtc.fit(iris.data, iris.target)
dtc.score(iris.data, iris.target)*100
# 과연 여기서 나온 100의 의미가 뭘까? 과연 이 100점이 이 모델이 완벽하다는 것을 의미할까?

- 따라서 훈련 데이터와 테스트 데이터를 분리해서 모델을 훈련시키고 정확도를 측정해야 함. 이를 교차 검증이라고 부름
  * 교차 검증을 통해 데이터의 편중을 막음
  * 교차 검증을 통해 과적합을 피하고 성능 저하를 막음
  * 일반적으로 교차 검증을 기반으로 1차 평가를 한 뒤 최종적으로 테스트 데이터 세트에 사용
- 이 과정을 도와주는 함수 중 하나가 바로 cross_validate()와 cross_vall_score()

In [11]:
from sklearn.model_selection import cross_val_score
cross_val_score(dtc, iris.data, iris.target, scoring='accuracy', cv=5)
# cross_val_score(모델, x 데이터, y 데이터, 평가 방법, 몇겹)

cross_val_score(dtc, iris.data, iris.target, scoring='accuracy', cv=5).mean()


0.9533333333333334

- 하지만 cross_val_score는 잘 쓰지 않음. 대신 교차검증과 최적 Hyperparameter 튜닝을 한번에 해주는 GridSearchCV 모듈을 사용

### GridSearchCV 클래스 : HyperParameter 튜닝 + 교차검증을 동시에 수행

#### decisiontreeclassifier에 gridsearchcv 적용하기

In [13]:
# Test / Train dataset 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(
    iris.data, iris.target, stratify=iris.target, test_size=0.2, random_state=2022
)

In [12]:
# 모델과 그에 해당하는 하이퍼 파라미터 확인
dtc=DecisionTreeClassifier(random_state=2022)
dtc.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 2022,
 'splitter': 'best'}

In [15]:
params={
    'max_depth' : [2,3,4,5,6],
    'min_samples_split' : [2,3,4],
}

In [16]:
from sklearn.model_selection import GridSearchCV
grid_dt=GridSearchCV(
    dtc, param_grid=params, scoring='accuracy', cv=5
)
# 이 조건 하에서는 총 75번 돌려서 최선의 모델을 찾아냄

In [17]:
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2022),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [19]:
# gridsearch를 이용해 산출한 best parameter 확인하기
grid_dt.best_params_

{'max_depth': 3, 'min_samples_split': 2}

In [21]:
# 최적 조건 하에서의 모델 정확도 측정
grid_dt.best_score_

0.9916666666666666

In [27]:
# 최적의 분류 모델 만들기
# dtc_best=DecisionTreeClassifier(max_depth=3, min_samples_split=2, random_state=2022) 코드의 심플한 표현
dtc_best=grid_dt.best_estimator_

In [28]:
# 최적의 분류 모델을 이용해 예측 및 평가
dtc_best.score(X_test, y_test)

0.9333333333333333

#### svm에 gridsearchcv 적용하기

In [29]:
from sklearn.svm import SVC
svc=SVC(random_state=2022)
svc.get_params()  # svm 모형에 적용되는 hyperparameter 확인하기

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 2022,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [30]:
params={'C' : [0.01,0.1,1,10,100]}
grid_sv=GridSearchCV(svc, params, scoring='accuracy', cv=5)

In [32]:
grid_sv.fit(X_train, y_train)
grid_sv.best_params_

{'C': 1}

In [39]:
# 최적의 c를 찾기 위해 c의 범위를 계속 좁혀나감
params={'C' : [0.2,0.6,1,4,8]}
grid_sv=GridSearchCV(svc, params, scoring='accuracy', cv=5)
grid_sv.fit(X_train, y_train)
grid_sv.best_params_

{'C': 4}

In [40]:
# 최적의 c를 찾기 위해 c의 범위를 계속 좁혀나감
params={'C' : [2,3,4,5,6]}
grid_sv=GridSearchCV(svc, params, scoring='accuracy', cv=5)
grid_sv.fit(X_train, y_train)
grid_sv.best_params_

{'C': 4}

In [41]:
svc_best=grid_sv.best_estimator_
svc_best.score(X_test, y_test)

0.9666666666666667